[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/jkanclerz/data-science-workshop-2021/blob/main/01--edi/07--xml-html.ipynb)

## Złożone formaty

* xml
* html

### XML

    * transfer danych / JPK / Dokumenty / Formularze
    * zagnieżdzone struktury
    * walidacja dokumentu
    * Ogólnodostępne parsery w językach programowania
    * Zorientowany biznesowo

```xml
<article>
    <info>
        <title>Some title</title>
        <authors>
            <author>
                <firstname>Jakub</firstname>
                <lastname>Kanclerz</lastname>
            </author>
        </authors>
    </info>
    <content>
        <section type="header">
            <paragraph>
                some title
            </paragraph>
        </section>
        <section type="main">
            <paragraph>
                some content
            </paragraph>
            <image src="image/src.jpeg"/>
        </section>
    </content>
</article>
```

### Html


```html
<li class="link">
    <div class="article" data-id="4789115">
        <a href="#4789115">
            <span>85</span>
            <span class="button button submit">wykop</span>
        </a>
    </div>

    <div class="media-content m-reset-float ">
        <a href="https://www.wykop.pl/link/4789115/w-jaki-sposob-facebook-stal-sie-lewicowa-sekta-opowiesc-pracownika-firmy/">
            <img src="https://www.wykop.pl/cdn/c3397993/link_MwUxfsmJdpfvuiy0ICl1l0Yw35VeWgBc,w207h139.jpg" alt="W jaki sposób Facebook stał się lewicową sektą? Opowieść pracownika firmy.">
        </a>
    </div>
    <div class="lcontrast m-reset-margin">
        <h2>
            <a href="https://www.wykop.pl/link/4789115/w-jaki-sposob-facebook-stal-sie-lewicowa-sekta-opowiesc-pracownika-firmy/"
                rel="nofollow noopener noreferrer" title="W jaki sposób Facebook stał się lewicową sektą? Opowieść pracownika firmy.">W
                jaki sposób Facebook stał się lewicową sektą? Opowieść pracownika firmy.</a>
        </h2>


        <div class="description">
            <p class="text">
                <a href="https://www.wykop.pl/link/4789115/w-jaki-sposob-facebook-stal-sie-lewicowa-sekta-opowiesc-pracownika-firmy/"
                    title="">
                    Tragiczna relacja pracownika, który przez lata obserwował jak serwis promujący wolność słowa
                    zamienił się w ideologiczny rynsztok prześladujący pracowników o zróżnicowanym światopoglądzie.
                </a>
            </p>
        </div>
    </div>
</li>
```

In [1]:
!pip install bs4

  Using cached bs4-0.0.1.tar.gz (1.1 kB)
  Preparing metadata (setup.py) ... done
  Using cached beautifulsoup4-4.10.0-py3-none-any.whl (97 kB)
Using legacy 'setup.py install' for bs4, since package 'wheel' is not installed.
    Running setup.py install for bs4 ... done


In [2]:
sample_xml = '''
<article>
    <info>
        <title>Some title</title>
        <authors>
            <author>
                <firstname>Jakub</firstname>
                <lastname>Kanclerz</lastname>
            </author>
        </authors>
    </info>
    <content>
        <section type="header">
            <paragraph>
                some title
            </paragraph>
        </section>
        <section type="main">
            <paragraph>
                some content
            </paragraph>
            <image src="image/src.jpeg"/>
        </section>
    </content>
</article>
'''

In [3]:
import requests
from bs4 import BeautifulSoup as bs
from itertools import chain

In [4]:
loaded = bs(sample_xml)

In [5]:
soups = loaded.select('author firstname')

In [6]:
print(soups[0].text)

Jakub


### Webscrapping

### Lista sklepów biedronka
[https://www.biedronka.pl/pl/sklepy/lista](https://www.biedronka.pl/pl/sklepy/lista)

In [172]:
import requests
import itertools
from collections import namedtuple

In [173]:
URL_TEMPLATE = "https://www.biedronka.pl/pl/sklepy/lista,page,{}"
PAGES_TO_SCAN = 10
#PAGES_TO_SCAN = 160

In [186]:
urls = [URL_TEMPLATE.format(p) for p in range(1, PAGES_TO_SCAN)]

In [187]:
responses = (requests.get(url) for url in urls)

In [188]:
contents = (r.text for r in responses)

In [189]:
soups = (bs(html_content) for html_content in contents)

In [190]:
Shop = namedtuple('Shop', ['city', 'zip', 'street'])

In [191]:
example_shop_element = '''
<h4 style="margin-bottom: 10px;">
    Rzeszów            	<span class="shopFullAddress">
        <span class="shopAddress">
            35-078<br>
            Hetmańska 56                    </span>
    </span>
</h4>
'''

In [192]:
def extract_shop(soup):
    try:
        city = (
            soup.select('h4')[0].text
                .strip()
                .split('\t')[0]
                .replace("  ", "")
        )
    except Error:
        city = None
    
    try:
        address = (soup.select('span.shopAddress')[0].text
            .strip()
            .replace("  ", "")
            .replace("|", " ")
            .replace('\r','')
            .split("\n")
        )
    except Error:
        address = [None, None]
    shop = Shop(city, address[0], address[1])
    return shop

In [193]:
assert extract_shop(bs(example_shop_element)) == Shop('Rzeszów', '35-078', 'Hetmańska 56')

In [194]:
extract_shop(bs(example_shop_element))

Shop(city='Rzeszów', zip='35-078', street='Hetmańska 56')

In [195]:
shop_soups_batch = (soup.select('ul.shopList li.shopListElement ') for soup in soups)

In [196]:
shop_soups = itertools.chain.from_iterable(shop_soups_batch)

In [197]:
shops = (extract_shop(soup) for soup in shop_soups)

In [198]:
shop_as_list = list(shops)

In [199]:
shop_as_list[:10]

[Shop(city='Rzeszów', zip='35-078', street='Hetmańska 56'),
 Shop(city='Kraśnik', zip='23-200', street='Urzędowska 472'),
 Shop(city='Medyka', zip='37-732', street='Przejście graniczne 405'),
 Shop(city='Police', zip='72-011', street='Głowackiego 10'),
 Shop(city='Szczecin', zip='70-781', street='Pomarańczowa 9'),
 Shop(city='Szczecin', zip='70-747', street='Karpia 1'),
 Shop(city='Myślibórz', zip='74-300', street='Lipowa 7'),
 Shop(city='Szczecin', zip='71-123', street='Stanisława Ignacego Witkiewicza 41'),
 Shop(city='Gorzów Wielkopolski', zip='66-400', street='Matejki 46'),
 Shop(city='Otwock', zip='05-400', street='Kołłątaja 61')]

## Utrwalenie rezultatów

In [200]:
import csv
with open('var/biedronkas.csv', 'w') as destination_file:
    writer = csv.writer(destination_file, delimiter=';', quotechar='"')
    for row in shop_as_list:
        writer.writerow(row)

In [204]:
!cat var/biedronkas.csv | head -n 5

In [205]:
import json
with open('var/biedronkas.data', 'w') as destination_file:
    for row in shop_as_list:
        destination_file.write(json.dumps(row._asdict()) + "\n")

In [206]:
!cat var/biedronkas.data | head -n 5

{"city": "Rzesz\u00f3w", "zip": "35-078", "street": "Hetma\u0144ska 56"}
{"city": "Kra\u015bnik", "zip": "23-200", "street": "Urz\u0119dowska 472"}
{"city": "Medyka", "zip": "37-732", "street": "Przej\u015bcie graniczne 405"}
{"city": "Police", "zip": "72-011", "street": "G\u0142owackiego 10"}
{"city": "Szczecin", "zip": "70-781", "street": "Pomara\u0144czowa 9"}


In [217]:
import pickle

In [220]:
with open('var/biedronkas.pickle', 'w') as f:
    pickle.dump(shop_as_list, f)

TypeError: write() argument must be str, not bytes

In [221]:
with open('var/biedronkas.pickle', 'wb') as f:
    pickle.dump(shop_as_list, f)

In [223]:
!cat var/biedronkas.pickle | head

Hetmańska 56�����h�Kraśnik��23-200��Urzędowska 472�����h�Medyka��37-732��Przejście graniczne 405�����h�Police��72-011��Głowackiego 10�����h�Szczecin��70-781��Pomarańczowa 9�����h�Szczecin��70-747��Karpia 1�����h�Myślibórz��74-300��Lipowa 7�����h�Szczecin��71-123��#Stanisława Ignacego Witkiewicza 41�����h�Gorzów Wielkopolski��66-400��
Grzybowska 61�����h�Łódź��91-446��Zgierska 15�����h�Łódź��91-843��Kowalska 2A�����h�Łódź��91-446��Zgierska 53�����h�Wrocław��54-002��Trzmielowicka 7/9�����h�Wrocław��54-155��Lotnicza 24�����h�Wrocław��54-017��Średzka 32�����h�Poznań��60-341��Marcelińska 77�����h�Poznań��60-771��
Matejki 62�����h�Poznań��60-162��
Szadecka 2�����h�Poznań��60-644��
Świnoujście��72-600��Nowokarsiborska 2�����h�Gostyń��63-800��
Łąkowa 3�����h�	Września��62-300��
Opieszyn 7�����h�Nowy Tomyśl��64-300��Witosa 7�����h�Jarocin��63-200��Powstańców Wielkopolskich 40�����h�Wolsztyn��64-200��Przemysłow

### Zadanie

Pobierz aktualne zainteresowania wykopowiczów (serwis wykop.pl)

In [210]:
urls = [
    "https://www.wykop.pl/strona/{}".format(page)
    for page in range(1, 10, 1)
]

In [211]:
import collections
from pprint import pprint as pp

Wykop = collections.namedtuple('Wykop', ['count', 'title', 'description', 'tags'])

def clear_text(text):
    return text.rstrip().replace("\n", '').replace("\t", '')

def grab_wykop_item_data(element):
    try:
        tags = map(lambda x: x.text, element.select('.article>div a.tag'))
        tags = filter(lambda x: x.startswith('#'), tags)
        
        return Wykop(
            int(element.select('.article>div a>span:first-child')[0].text),
            element.select('.article>div h2 a')[0]['title'],
            clear_text(element.select('.article>div .description')[0].text),
            list(tags))
    except Exception as e:
        return None

In [214]:
pages = (requests.get(url) for url in urls)
contents = (page.content for page in pages)

## Feel free to continue 

In [215]:
next(contents)

b'<!DOCTYPE HTML>\n<html>\n<head>\n\t<meta charset="UTF-8">\n\t<meta name="viewport" content="width=device-width, initial-scale=1.0">\n\t<title>Wykop.pl - newsy, aktualno\xc5\x9bci, gry, wiadomo\xc5\x9bci, muzyka, ciekawostki, filmiki</title>\n\t<meta name="robots" content="follow, index"/>\n\t<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n\t<meta http-equiv="Content-Language" content="pl"  />\n\t<meta name="description" content="Wykop jest miejscem, gdzie gromadzimy najciekawsze informacje z Sieci: newsy, artyku\xc5\x82y, linki. O tre\xc5\x9bci serwisu decyduj\xc4\x85 tylko i wy\xc5\x82\xc4\x85cznie nasi u\xc5\xbcytkownicy, dodaj\xc4\x85c newsy, komentuj\xc4\x85c i g\xc5\x82osuj\xc4\x85c na nie." />\n\t<meta name="keywords" content="Newsy, Aktualno\xc5\x9bci, Gry komputerowe, Super filmiki, Wiadomo\xc5\x9bci, Ciekawostki, Filmiki z humorem, \xc5\x9bmieszne filmy, Muzyka na codzie\xc5\x84" />\n\t<meta name="verify-v1" content="" />\n\t<meta name="google-site-ver